<a href="https://colab.research.google.com/github/whossssssss/Lazy-tourist/blob/main/Lazy_tourist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ipywidgets as widgets
from IPython.display import display
import requests
import json
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

class LazyTouristApp:
    def __init__(self):
        # DeepSeek API configuration
        self.api_key = ""
        self.api_url = "https://api.deepseek.com/v1/chat/completions"

        # Initialize variables
        self.current_city = ""
        self.attractions_data = []

        # Create UI elements
        self.create_widgets()

    def create_widgets(self):
        # City input
        self.city_input = widgets.Text(
            placeholder='Enter city name (e.g., Moscow, Paris)',
            description='City:',
            disabled=False
        )

        # Search button
        self.search_button = widgets.Button(
            description='Search Attractions',
            disabled=False,
            button_style=''
        )
        self.search_button.on_click(self.search_attractions)

        # Attractions list (multi-select)
        self.attractions_list = widgets.SelectMultiple(
            options=[],
            description='Attractions:',
            disabled=False,
            layout={'width': '500px', 'height': '200px'}
        )

        # Route button
        self.route_button = widgets.Button(
            description='Generate Route Plan',
            disabled=False,
            button_style=''
        )
        self.route_button.on_click(self.generate_route)

        # Output area
        self.output = widgets.Output()

        # Display UI
        display(widgets.VBox([
            widgets.HBox([self.city_input, self.search_button]),
            self.attractions_list,
            self.route_button,
            self.output
        ]))

    def call_deepseek_api(self, prompt):
        """Call DeepSeek API"""
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": "deepseek-chat",
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "temperature": 0.7
        }

        try:
            response = requests.post(self.api_url, headers=headers, json=payload)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            with self.output:
                print(f"API Error: {str(e)}")
            return None

    def get_city_coordinates(self, city_name):
        """Get city coordinates using geopy"""
        geolocator = Nominatim(user_agent="lazy_tourist_app")
        try:
            location = geolocator.geocode(city_name, timeout=10)
            if location:
                return (location.latitude, location.longitude)
            return None
        except GeocoderTimedOut:
            with self.output:
                print(f"Geocoding service timed out for {city_name}")
            return None
        except Exception as e:
            with self.output:
                print(f"Geocoding error: {str(e)}")
            return None

    def search_attractions(self, button):
        city = self.city_input.value.strip()
        if not city:
            with self.output:
                print("Error: Please enter a city name")
            return

        with self.output:
            print(f"Searching attractions in {city}...")

        try:
            # Get city coordinates first
            coordinates = self.get_city_coordinates(city)
            if not coordinates:
                with self.output:
                    print(f"Could not locate {city}. Please check the spelling or try a nearby major city.")
                return

            # Use DeepSeek API to get attractions
            prompt = f"""List the top 5 tourist attractions in {city} with the following requirements:
1. Return PURE JSON only, no Markdown or additional text
2. Include these fields for each: name, rating (1-5), address, description, lat, lng
3. Provide information in English
4. Example format:
[
  {{
    "name": "Attraction Name",
    "rating": 4.5,
    "address": "Full Address",
    "description": "Brief description",
    "lat": 12.3456,
    "lng": 78.9012
  }}
]"""
            api_response = self.call_deepseek_api(prompt)

            if not api_response or 'choices' not in api_response:
                with self.output:
                    print("Error: Failed to get attraction data")
                return

            # Parse API response
            content = api_response['choices'][0]['message']['content']

            # Clean the response
            content = content.replace('```json', '').replace('```', '').strip()

            try:
                attractions = json.loads(content)
            except json.JSONDecodeError as e:
                with self.output:
                    print(f"JSON Parse Error: {str(e)}")
                    print(f"Raw content: {content}")
                return

            # Validate data format
            if not isinstance(attractions, list):
                with self.output:
                    print("Error: API didn't return a list")
                return

            required_fields = ['name', 'rating', 'address', 'description', 'lat', 'lng']
            for attr in attractions:
                if not all(field in attr for field in required_fields):
                    with self.output:
                        print("Error: Missing required fields in attraction data")
                    return

            # Update attractions list
            self.attractions_data = attractions
            self.attractions_list.options = [(f"{attr['name']} (Rating: {attr['rating']})", idx)
                                          for idx, attr in enumerate(attractions)]

            self.current_city = city
            with self.output:
                print(f"Found {len(attractions)} attractions in {city}")

        except Exception as e:
            with self.output:
                print(f"Search Error: {str(e)}")

    def generate_route(self, button):
        """Generate multi-attraction route plan"""
        if not self.current_city:
            with self.output:
                print("Error: Please search for a city first")
            return

        selected_indices = self.attractions_list.value
        if not selected_indices:
            with self.output:
                print("Error: Please select at least one attraction")
            return

        # Get selected attractions
        selected_attractions = [self.attractions_data[idx] for idx in selected_indices]

        with self.output:
            print(f"Planning optimal route for {len(selected_attractions)} attractions...")

        # Prepare route planning prompt
        attractions_list = "\n".join([f"- {attr['name']} ({attr['address']})" for attr in selected_attractions])
        prompt = f"""I'm traveling in {self.current_city} and want to visit these attractions:
{attractions_list}

Please create an efficient sightseeing route considering:
1. Geographical proximity between attractions
2. Logical visiting order (morning to evening)
3. Recommended time at each location (based on size/popularity)
4. Transportation options between locations (metro/bus/taxi/walking) with time estimates
5. Reasonable breaks/lunch time

Provide the plan in this format:
1. First Stop: [Attraction Name]
   - Suggested duration: [time]
   - To next stop: [transport] ([time])
2. Second Stop: [Attraction Name]
   ...

Include total estimated sightseeing time (from start to finish) at the end.
Respond in English only."""

        # Call DeepSeek API for route planning
        api_response = self.call_deepseek_api(prompt)
        if not api_response or 'choices' not in api_response:
            with self.output:
                print("Error: Failed to generate route")
            return

        # Display route advice
        route_advice = api_response['choices'][0]['message']['content']
        with self.output:
            print("\n=== OPTIMIZED ROUTE PLAN ===")
            print(route_advice)

# Create and run the app
app = LazyTouristApp()